In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 
import matplotlib.pyplot as plt
import os
import decimal

In [ ]:
def get_html_files():

    url = 'https://www.basketball-reference.com/leagues/' 

    seasons = ['NBA_2011_per_game.html', 'NBA_2012_per_game.html', 'NBA_2013_per_game.html', 'NBA_2014_per_game.html', 'NBA_2015_per_game.html',
            'NBA_2016_per_game.html', 'NBA_2017_per_game.html', 'NBA_2018_per_game.html', 'NBA_2019_per_game.html', 'NBA_2020_per_game.html',
            'NBA_2021_per_game.html', 'NBA_2022_per_game.html', 'NBA_2023_per_game.html', 'NBA_2024_per_game.html']
    for season in seasons:
        base_url = url + season
        dir_path = '/Users/josephcrawford/Documents/data_engineering/test'

        os.makedirs(dir_path, exist_ok=True)
        try:
            response = requests.get(base_url)
        except requests.exceptions.RequestException as e:
            print('HTTP Error')
            print(e)
        with open(os.path.join(dir_path, season) ,'w', encoding='utf-8') as file:
                file.write(response.text)
get_html_files()

In [ ]:
def parse_html(file_path, season):
   
    with open(file_path, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'html.parser')

    tables = soup.find_all('table')
    dataframes = []

    for table in tables:
        dfs = pd.read_html(str(table))
        for df in dfs:
            df['SEASON'] = season
            dataframes.extend(dfs)

    return dataframes

In [ ]:

def process_htmls(working_directory):   
    processed_dataframes = []
    
    for file in os.listdir(working_directory):
        if file.endswith('.html'):
            file_path = os.path.join(working_directory, file)
            season = file.split('_')[1]
            dataframes = parse_html(file_path, season)
            processed_dataframes.extend(dataframes)
   

    combined_dfs = pd.concat(processed_dataframes, ignore_index=False)

    return combined_dfs


working_directory = '/Users/josephcrawford/Documents/data_engineering/NBA_Stats' 

combined_dfs = process_htmls(working_directory)

In [ ]:
df2 = combined_dfs[~combined_dfs['Rk'].str.contains('Rk')]
df2 = df2.drop('Rk', axis=1)